In [0]:
import tensorflow as tf

In [2]:
tf.__version__

'2.2.0'

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Hashing")
!pwd

/content/gdrive/My Drive/Colab Notebooks/Hashing


In [0]:
train_x_path = "./gen_data/train_X"
train_y_path = "./gen_data/train_Y"
test_x_path = "./gen_data/test_X"

In [0]:
import numpy as np
def load_dataset():
  train_X = np.loadtxt(train_x_path)
  train_Y = np.loadtxt(train_y_path)
  test_X = np.loadtxt(test_x_path)
  train_X.dtype = 'float64'
  train_Y.dtype = 'float64'
  test_X.dtype = 'float64'
  return train_X, train_Y, test_X

In [0]:
train_X,train_Y,test_X = load_dataset()

In [0]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [11]:
from gensim.models.word2vec import Word2Vec

wv_model = Word2Vec.load("./gen_data/word2vec.model")
embedding_matrix = wv_model.wv.vectors
embedding_matrix

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


array([[ 5.78204274e-01,  6.03120551e-02,  3.60607475e-01, ...,
        -3.83604765e-01, -2.35126719e-01, -3.39127541e-01],
       [ 2.48232469e-01, -1.09330297e-01, -1.07853085e-01, ...,
        -4.05686855e-01, -3.84583443e-01,  8.72177184e-01],
       [-2.61794060e-01,  1.58479378e-01, -3.25242400e-01, ...,
        -1.04319692e+00, -3.70189011e-01, -6.24906011e-02],
       ...,
       [ 3.13954473e-01,  6.27483010e-01,  8.70058371e-04, ...,
        -6.59883201e-01,  1.04633875e-01,  5.38641512e-02],
       [-3.03027540e-01, -6.11446559e-01, -1.08515084e+00, ...,
         1.52157664e-01,  3.57411057e-01, -8.81655991e-01],
       [ 3.11857313e-01, -5.35719246e-02, -3.20000172e-01, ...,
        -1.04822442e-01,  1.92798018e-01, -4.42412287e-01]], dtype=float32)

In [12]:
embedding_matrix.shape

(32909, 300)

In [0]:
np.savetxt("./gen_data/embedding_matrix", embedding_matrix, fmt='%0.8f')

In [0]:
def seq2seq(input_length, output_sequence_length, embedding_matrix, vocab_size):
  model = Sequential()
  model.add(Embedding(input_dim=vocab_size, output_dim=300, weights=[embedding_matrix], trainable=False,
                      input_length=input_length))
  model.add(Bidirectional(GRU(300, return_sequences=False)))
  model.add(Dense(300, activation="relu"))
  model.add(RepeatVector(output_sequence_length))
  model.add(Bidirectional(GRU(300, return_sequences=True)))
  model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))
  model.compile(loss=sparse_categorical_crossentropy,
                optimizer=Adam(1e-3))
  model.summary()
  return model

In [15]:
train_X.shape

(82871, 260)

In [0]:
def load_vocab(file_path):
  vocab = {}
  reverse_vocab = {}
  for line in open(file_path, "r", encoding='utf-8').readlines():
      word, index = line.strip().split("\t")
      index = int(index)
      vocab[word] = index
      reverse_vocab[index] = word
  return vocab, reverse_vocab

In [0]:
vocab, reverse_vocab = load_vocab("./gen_data/vocab.txt")

In [0]:
# 输入的长度 x max_len
input_length = train_X.shape[1]
# 输出的长度 y max_len
output_sequence_length = train_Y.shape[1]
# 词表大小
vocab_size=len(vocab)
# 词向量矩阵
embedding_matrix = wv_model.wv.vectors

In [19]:
model = seq2seq(input_length,output_sequence_length,embedding_matrix,vocab_size)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 260, 300)          9872700   
_________________________________________________________________
bidirectional (Bidirectional (None, 600)               1083600   
_________________________________________________________________
dense (Dense)                (None, 300)               180300    
_________________________________________________________________
repeat_vector (RepeatVector) (None, 33, 300)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 33, 600)           1083600   
_________________________________________________________________
time_distributed (TimeDistri (None, 33, 32909)         19778309  
Total params: 31,998,509
Trainable params: 22,125,809
Non-trainable params: 9,872,700
____________________________________

In [21]:
model.fit(train_X, train_Y, batch_size=32, epochs=1, validation_split=0.2)

2072/2072 [==============================] - 657s 317ms/step - loss: 3.1931 - val_loss: 2.9219


In [22]:
train_X.shape

(82871, 260)

In [0]:
model.fit(train_X, train_Y, batch_size=16, epochs=5, validation_split=0.2)

Epoch 1/5
1041/4144 [======>.......................] - ETA: 11:17 - loss: 2.9196